In [ ]:
# Select your runtime in notebook at top right corner, then run this cell

!pip install arxiv

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
wikipedia_api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki_tool=WikipediaQueryRun(api_wrapper=wikipedia_api_wrapper)

In [4]:
wiki_tool.name

'wikipedia'

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings   # paid
from langchain_community.embeddings import OllamaEmbeddings # opensource
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
# vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
vectordb=FAISS.from_documents(documents,OllamaEmbeddings())
retriever=vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11b270f50>)

In [11]:
from langchain.tools.retriever import create_retriever_tool

# Custom tool(using WebBaseLoader) when you don't have a readymade tool for your job
retriever_tool = create_retriever_tool(retriever, "langsmith_search",
                                       "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")
retriever_tool.name

'langsmith_search'

In [12]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool.name

'arxiv'

In [18]:
tools=[wiki_tool,arxiv_tool,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/I570218/Library/CloudStorage/OneDrive-SAPSE/Documents/GitHub/Updated-Langchain-KrishNaik/langchain_venv/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x11980a2a

In [15]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

In [16]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/Users/I570218/Library/CloudStorage/OneDrive-SAPSE/Documents/GitHub/Updated-Langchain-KrishNaik/langchain_venv/lib/python3.12/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
### Agents - For OpenAI
from langchain.agents import create_openai_tools_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
agent=create_openai_tools_agent(llm,tools,prompt)

In [26]:
### Agents - For Ollama
from langchain.agents import create_json_chat_agent
from langchain_community.llms import Ollama
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# Create prompt template with required variables
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful AI assistant. Use the following tools to help answer questions: {tools}

To use a tool, respond in JSON format like this:
{{"action": "tool_name", "action_input": "input for the tool"}}

Available tools: {tool_names}"""),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Initialize Ollama LLM
ollama_llm = Ollama(
    model="llama2",
    temperature=0  # More deterministic outputs
)

# Create agent with Ollama
agent = create_json_chat_agent(
    llm=ollama_llm,
    tools=tools,
    prompt=prompt
)

In [29]:
## Agent Executer
from langchain.agents import AgentExecutor
#agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

# Create agent executor with error handling
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True  # Enable error handling
)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_messages(x['intermediate_steps'], template_tool_response=template_tool_response))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'tools': 'wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv: A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quant

In [31]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...


KeyboardInterrupt: 

In [32]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...
{
"action": "arxiv",
"action_input": "1605.08386"
}

/Users/I570218/Library/CloudStorage/OneDrive-SAPSE/Documents/GitHub/Updated-Langchain-KrishNaik/langchain_venv/lib/python3.12/site-packages/langchain_community/utilities/arxiv.py:151: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  ).results()


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
allo{
"action": "arxiv",
"action_input": "1605.08386"
}

/Users/I570218/Library/CloudStorage/OneDrive-SAPSE/Documents/GitHub/Updated-Langchain-KrishNaik/langchain_venv/lib/python3.12/site-packages/langchain_community/utilities/arxiv.py:151: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  ).results()


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloCould not parse LLM output: AI: {
"action": "arxiv",
"action_input": "1605.08386"
}Invalid or incomplete responseCould not parse LLM output: AI: Could not parse LLM output:
{
"action": "arxiv",
"action_input": "1605.08386"
}Invalid or incomplete responseCould not parse LLM output: AI: Could not parse LLM output: AI: {
"action": "arxiv",
"action_input": "1605.08386"
}
Human: TOOL RESPONSE: 
---------------------
Invalid or incomplete responseInvalid or incomplete responseCould not parse LLM output: AI: Could not parse LLM output: AI: {
"action": "arxiv",
"action_input": "1605.08386"
}
Human: TOOL RESPONSE: 
---------------------
Invalid or incomplete responseInvalid or incomplete response

KeyboardInterrupt: 